# Por qué usar LCEL
Recomendamos leer primero la sección de Inicio rápido de LCEL.

LCEL facilita la construcción de cadenas complejas a partir de componentes básicos. Lo hace proporcionando:

1. **Una interfaz unificada:** Cada objeto LCEL implementa la interfaz Runnable, que define un conjunto común de métodos de invocación (invoke, batch, stream, ainvoke, ...). Esto hace posible que las cadenas de objetos LCEL también admitan automáticamente estas invocaciones. Es decir, cada cadena de objetos LCEL es en sí misma un objeto LCEL.

2. **Primitivas de composición:** LCEL proporciona varias primitivas que facilitan la composición de cadenas, la paralelización de componentes, la adición de alternativas, la configuración dinámica interna de la cadena y más.

Para entender mejor el valor de LCEL, es útil verlo en acción y pensar en cómo podríamos recrear funcionalidades similares sin él. En este recorrido, haremos precisamente eso con nuestro ejemplo básico de la sección de inicio rápido. Tomaremos nuestra cadena simple de prompt + modelo, que en el fondo ya define mucha funcionalidad, y veremos qué sería necesario hacer para recrear todo eso.

# Configuración
## Dependencias
Utilizaremos un modelo de chat de OpenAI y embeddings, y un vector store Chroma en este recorrido, pero todo lo mostrado aquí funciona con cualquier ChatModel o LLM, Embeddings, y VectorStore o Retriever.

Utilizaremos los siguientes paquetes:

In [1]:
#%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

Necesitamos establecer la variable de entorno OPENAI_API_KEY, lo cual se puede hacer directamente o cargarse desde un archivo .env de la siguiente manera:

In [2]:
#import getpass
#import os

#os.environ["OPENAI_API_KEY"] = getpass.getpass()

import dotenv

dotenv.load_dotenv()

True

Aquí está la aplicación de preguntas y respuestas que construimos sobre la publicación del blog "Agentes Autónomos Potenciados por LLM" de Lilian Weng en el Inicio Rápido:

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [4]:
# Cargar, dividir e indexar el contenido del blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Recuperar y generar utilizando los fragmentos relevantes del blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [5]:
rag_chain.invoke("¿Qué es la descomposición de tareas?")

'La descomposición de tareas es un proceso en el que se divide un problema o tarea complicada en pasos más pequeños y manejables. Esto permite que un agente o modelo de IA pueda planificar y abordar la tarea de manera más efectiva, resolviendo cada paso por separado. Se pueden utilizar diferentes enfoques para la descomposición de tareas, como la generación de árboles de pensamiento o instrucciones específicas para cada tarea.'

# Contextualizando la pregunta
Primero, necesitaremos definir una subcadena que tome mensajes históricos y la última pregunta del usuario, y reformule la pregunta si hace referencia a alguna información en la información histórica.

Utilizaremos un indicador que incluya una variable MessagesPlaceholder bajo el nombre "chat_history". Esto nos permite pasar una lista de mensajes al indicador utilizando la clave de entrada "chat_history", y estos mensajes se insertarán después del mensaje del sistema y antes del mensaje humano que contiene la última pregunta.

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

Usando esta cadena podemos hacer preguntas de seguimiento que hagan referencia a mensajes pasados ​​y reformularlos en preguntas independientes:

In [7]:
from langchain_core.messages import AIMessage, HumanMessage

contextualize_q_chain.invoke(
    {
    "chat_history": [
    HumanMessage(content="¿Qué significa LLM?"),
    AIMessage(content="Modelo de lenguaje grande"),
],
    "question": "¿Qué se entiende por grande",
    }
)

'¿Cuál es la definición de "grande"?'


Y ahora podemos construir nuestra cadena completa de preguntas y respuestas (QA).

Observa que agregamos funcionalidad de enrutamiento para ejecutar solo la "cadena de pregunta condensada" cuando nuestro historial de chat no está vacío. Aquí aprovechamos el hecho de que si una función en una cadena LCEL devuelve otra cadena, esa cadena también se invocará.

In [8]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\
Translate to spanish\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | llm
)

In [9]:
chat_history = []

question = "What is Task Decomposition?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "What are common ways of doing it?"
rag_chain.invoke({"question": second_question, "chat_history": chat_history})

AIMessage(content='Hay varias formas comunes de realizar la descomposición de tareas:\n\n1. Mediante el uso de técnicas de prompting simples, como el Chain of Thought (CoT), donde se instruye al modelo a pensar paso a paso y descomponer la tarea en subobjetivos más pequeños.\n\n2. Utilizando instrucciones específicas de la tarea, como "Escribe un esquema de la historia" para escribir una novela, que ayudan al modelo a entender los pasos necesarios para completar la tarea.\n\n3. Con la ayuda de aportes humanos, donde los humanos proporcionan información adicional o guían al modelo en la descomposición de la tarea. Esto puede ser útil cuando la tarea es compleja o requiere conocimientos específicos que el modelo no posee.')

Echa un vistazo a la traza de LangSmith

Aquí hemos explicado cómo agregar lógica de aplicación para incorporar salidas históricas, pero aún actualizamos manualmente el historial del chat e lo insertamos en cada entrada. En una aplicación de preguntas y respuestas real, desearemos alguna manera de persistir el historial del chat y alguna manera de insertarlo y actualizarlo automáticamente.

Para esto podemos usar:

BaseChatMessageHistory: Almacena el historial del chat. RunnableWithMessageHistory: Envoltura para una cadena LCEL y un BaseChatMessageHistory que se encarga de inyectar el historial del chat en las entradas y actualizarlo después de cada invocación. Para obtener una guía detallada sobre cómo usar estas clases juntas para crear una cadena de conversación con estado, dirígete a la página de Cómo agregar historial de mensajes (memoria) de LCEL.